In [ ]:
from langgraph.graph import StateGraph, START, END, add_messages
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage, SystemMessage
from langgraph.prebuilt import ToolNode

from utils import setup_logger
from components.llm import get_llm
from components.tools import create_driver, access_url

from template import planner_prompt, system_prompt, create_driver_prompt
from utils import dump_json_to_file

from typing_extensions import TypedDict, Annotated
from pydantic import BaseModel, Field
import json
import operator


logger = setup_logger("agent")

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    driver: object
    url: str
    request: str
    response: str

In [18]:
state: State = {
        "messages": [],
        "driver": None,
        "url": "http://bing.com",
        "request": "Search for the term 'LangGraph' on the page and provide a summary of the content found.",
        "response": "",

}

In [ ]:
llm = get_llm()

user_prompt = f"""Your first task is to create a web driver instance using the create_driver tool. \
Make sure to use the tool correctly to initialize the driver before performing any other actions."""

messages = [
    HumanMessage(content=user_prompt)
]

llm_with_tools = llm.bind_tools([create_driver])

chain = create_driver_prompt | llm_with_tools

try:
    response = chain.invoke(
        {
            "request": messages[0].content
        }
    )
    logger.info("Web driver created successfully.")
except Exception as e:
    logger.error(f"Error creating web driver: {e}")
    state["driver"] = None

In [20]:
dump_json_to_file(dict(response), "response.json")

In [25]:
response.tool_calls

[{'name': 'create_driver',
  'args': {},
  'id': 'f5dc9b40-a3d1-41ec-af1a-9ba771a97bd0',
  'type': 'tool_call'}]

In [ ]:
# from langgraph.prebuilt import ToolNode



# tool_node = ToolNode([create_driver])

# state_with_tool = {
#     "messages": messages + [response],
# }

# tool_result = tool_node.invoke(state_with_tool)
# print(tool_result)

{'messages': [ToolMessage(content='<selenium.webdriver.chrome.webdriver.WebDriver (session="f93896393b81ecb6c532c95652bf675a")>', name='create_driver', tool_call_id='f5dc9b40-a3d1-41ec-af1a-9ba771a97bd0')]}


In [2]:
llm = get_llm()

In [3]:
tools = [create_driver, access_url]
tool_node = ToolNode(tools)
llm_with_tools = llm.bind_tools(tools)

response1 = llm_with_tools.invoke([HumanMessage("Create a web driver")])
result1 = tool_node.invoke({"messages": [response1]})
print(result1["messages"][0].content)

response2 = llm_with_tools.invoke([HumanMessage("Access the URL http://bing.com")])
result2 = tool_node.invoke({"messages": [response2]})
print(result2["messages"][0].content)


{"message": "WebDriver instance created."}
null
